# retirement calculation

given: life expectancy, current age, retirement age, account value at retirement (how much money you have when you retire, at the time of your retirement), expenditure after retirement (how much money you need to spend, at today's value before inflation)

simulate: inflation, investment return. three ways of number generation (1) assumed constant (2) normal distribution randomiser based on historical returns within a pre-defined time range (2) bootstrapping from historical returns within a pre-defined time range  

calculate: success rate (success = account never dip below safety threshold throughout lifetime), life-end account value distribution

assume: no major change of lifestyle during retirement that requires deviation from usual expenditure pattern. portfolio stays at the same composition (monthly rebalanced)

In [1]:
import sys
project_root = r"/Users/cococao/Desktop/retirement_calculator"
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
from pathlib import Path
from dotenv import load_dotenv
# locate .env (prefer project root, fall back to current working directory)
dotenv_path = Path(project_root) / ".env"
if not dotenv_path.exists():
    dotenv_path = Path(".") / ".env"

if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path, override=False)

In [2]:
retirement_capital = 1000000
retirement_age = 40

return_method = 'normal'
proportion_stock = 0.6
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000 # the account balance below which the simulation is considered a failure if any month falls below it

In [3]:
# for after retirement 
from utilities.simulators.drawdown.paralle_worker_simulator import ParallelWorkerSimulator

pws = ParallelWorkerSimulator()
pws.simulate(retirement_capital=retirement_capital,retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=proportion_stock, threshold_failure=threshold_failure)

{'final_account_analysis': {'mean': np.float64(3814238.24565166),
  'median': np.float64(3579044.3800150636),
  'percentile_10': np.float64(1600731.1497368359),
  'percentile_90': np.float64(6337229.982331937)},
 'failure_rate': np.float64(0.0032)}

# whole-path calculation
including the accumulative phase (saving before retirement). to calculate how early I can retire at what income & expenditure level.

given: life expectancy, current age, retirement age, current account value (how much money you have now), current expenditure (at today's value, before inflation), post-retirement expenditure (at today's value, before inflation), average monthly income during accumulation phase, average annual income increment during accumulation phase

simulate: inflation, investment return. three ways of number generation (1) assumed constant (2) normal distribution randomiser based on historical returns within a pre-defined time range (2) bootstrapping from historical returns within a pre-defined time range  

calculate: success rate (success = account never dip below safety threshold throughout lifetime), life-end account value distribution

assume: no lifestyle inflation during accumulation phase. no major unexpected expenditure during retirement. portfolio stays at the same composition (monthly rebalanced). constant pre-defined annual income increment rate during accumulation phase.

In [4]:
# overall, with accumulation

monthly_net_income = 6000
retirement_age = 40

return_method = 'normal'
proportion_stock = 1
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000


In [5]:
from utilities.simulators.whole_path.paralle_worker_simulator import ParallelWorkerSimulator
pws = ParallelWorkerSimulator()
pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=proportion_stock, threshold_failure=threshold_failure)

{'final_account_analysis': {'mean': np.float64(7567148.69445888),
  'median': np.float64(6615211.28563899),
  'percentile_10': np.float64(2133480.489736402),
  'percentile_90': np.float64(14215319.457051078)},
 'failure_rate': np.float64(0.0122)}

# Asset allocation tester

We want to (1) retire early (2) reduce risk of account failure in lifetime to reasonably low levels. Given our customised set of assumptions about , here I test out a few strategies of asset allocation to see which one suits our specific use case better.

strategies tested:
- constant (monthly rebalanced)
- glidepath (100% stocks until retirement. after retirement, gradually reduce stocks composition until it reaches zero at the end of life expectancy)
- annual goal-fitter: set a goal of account value to reach by end of each year until end of life expectancy. at the end of year X, depending on account value, adjust the allocation so that (1) the possibility of the account value reaching goal at the end of year (X+1) does not fall below a pre-defined threshold (2) for all possible allocations that satisfy the first condition, choose the allocation with highest median account value (among all simulated situations for year (X+1))


In [6]:
# overall, with accumulation
monthly_net_income = 7000
retirement_age = 35

return_method = 'normal'
start_year = 1962 # earliest data for historical investment return is from 1928
end_year = 2024
n_simulation = 5000
inflation_country = "mixed" # data available for US, UK, singapore, spain. mixed averages over all available countries
threshold_failure=10000


In [7]:
from utilities.simulators.whole_path.paralle_worker_simulator import ParallelWorkerSimulator
pws = ParallelWorkerSimulator()

dic_testing_strategies_results = {}

for k in range(11):
    stock_proportion = round(k/10,1)
    testing_strategy = ('constant', 'stock_proportion = ' + str(stock_proportion))
    res = pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, proportion_stock=stock_proportion, threshold_failure=threshold_failure)
    dic_testing_strategies_results[testing_strategy] = res

for glidepath_type in ['linear', 'concave_down', 'concave_up']:
    dic_testing_strategies_results[('glidepath', glidepath_type)] = pws.simulate(monthly_net_income=monthly_net_income, retirement_age=retirement_age,n_simulation=n_simulation, start_year=start_year, end_year=end_year, return_method=return_method, threshold_failure=threshold_failure, f_portfolio_composition = True, portfolio_method = 'glidepath', glidepath_type = glidepath_type)

TypeError: GlidePathCompositor._y_concave_down() got multiple values for argument 'p'

In [ ]:
for method in dic_testing_strategies_results.keys():
    if method[0] == 'glidepath':
        print(method)
        print(dic_testing_strategies_results[method])

{'final_account_analysis': {'mean': np.float64(1830546.320186502),
  'median': np.float64(1446779.6157871513),
  'percentile_10': np.float64(-1104785.9874163156),
  'percentile_90': np.float64(5229094.842301678)},
 'failure_rate': np.float64(0.2494)}